In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
torch.manual_seed(1)

In [ ]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [ ]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [ ]:
test_sentence

['When',
 'forty',
 'winters',
 'shall',
 'besiege',
 'thy',
 'brow,',
 'And',
 'dig',
 'deep',
 'trenches',
 'in',
 'thy',
 "beauty's",
 'field,',
 'Thy',
 "youth's",
 'proud',
 'livery',
 'so',
 'gazed',
 'on',
 'now,',
 'Will',
 'be',
 'a',
 "totter'd",
 'weed',
 'of',
 'small',
 'worth',
 'held:',
 'Then',
 'being',
 'asked,',
 'where',
 'all',
 'thy',
 'beauty',
 'lies,',
 'Where',
 'all',
 'the',
 'treasure',
 'of',
 'thy',
 'lusty',
 'days;',
 'To',
 'say,',
 'within',
 'thine',
 'own',
 'deep',
 'sunken',
 'eyes,',
 'Were',
 'an',
 'all-eating',
 'shame,',
 'and',
 'thriftless',
 'praise.',
 'How',
 'much',
 'more',
 'praise',
 "deserv'd",
 'thy',
 "beauty's",
 'use,',
 'If',
 'thou',
 'couldst',
 'answer',
 "'This",
 'fair',
 'child',
 'of',
 'mine',
 'Shall',
 'sum',
 'my',
 'count,',
 'and',
 'make',
 'my',
 'old',
 "excuse,'",
 'Proving',
 'his',
 'beauty',
 'by',
 'succession',
 'thine!',
 'This',
 'were',
 'to',
 'be',
 'new',
 'made',
 'when',
 'thou',
 'art',
 'old,',
 

In [ ]:
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]

ngrams

[(['forty', 'When'], 'winters'),
 (['winters', 'forty'], 'shall'),
 (['shall', 'winters'], 'besiege'),
 (['besiege', 'shall'], 'thy'),
 (['thy', 'besiege'], 'brow,'),
 (['brow,', 'thy'], 'And'),
 (['And', 'brow,'], 'dig'),
 (['dig', 'And'], 'deep'),
 (['deep', 'dig'], 'trenches'),
 (['trenches', 'deep'], 'in'),
 (['in', 'trenches'], 'thy'),
 (['thy', 'in'], "beauty's"),
 (["beauty's", 'thy'], 'field,'),
 (['field,', "beauty's"], 'Thy'),
 (['Thy', 'field,'], "youth's"),
 (["youth's", 'Thy'], 'proud'),
 (['proud', "youth's"], 'livery'),
 (['livery', 'proud'], 'so'),
 (['so', 'livery'], 'gazed'),
 (['gazed', 'so'], 'on'),
 (['on', 'gazed'], 'now,'),
 (['now,', 'on'], 'Will'),
 (['Will', 'now,'], 'be'),
 (['be', 'Will'], 'a'),
 (['a', 'be'], "totter'd"),
 (["totter'd", 'a'], 'weed'),
 (['weed', "totter'd"], 'of'),
 (['of', 'weed'], 'small'),
 (['small', 'of'], 'worth'),
 (['worth', 'small'], 'held:'),
 (['held:', 'worth'], 'Then'),
 (['Then', 'held:'], 'being'),
 (['being', 'Then'], 'asked

In [ ]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}
vocab

{"'This",
 'And',
 'How',
 'If',
 'Proving',
 'Shall',
 'Then',
 'This',
 'Thy',
 'To',
 'Were',
 'When',
 'Where',
 'Will',
 'a',
 'all',
 'all-eating',
 'an',
 'and',
 'answer',
 'art',
 'asked,',
 'be',
 'beauty',
 "beauty's",
 'being',
 'besiege',
 'blood',
 'brow,',
 'by',
 'child',
 'cold.',
 'couldst',
 'count,',
 'days;',
 'deep',
 "deserv'd",
 'dig',
 "excuse,'",
 'eyes,',
 'fair',
 "feel'st",
 'field,',
 'forty',
 'gazed',
 'held:',
 'his',
 'in',
 'it',
 'lies,',
 'livery',
 'lusty',
 'made',
 'make',
 'mine',
 'more',
 'much',
 'my',
 'new',
 'now,',
 'of',
 'old',
 'old,',
 'on',
 'own',
 'praise',
 'praise.',
 'proud',
 'say,',
 'see',
 'shall',
 'shame,',
 'small',
 'so',
 'succession',
 'sum',
 'sunken',
 'the',
 'thine',
 'thine!',
 'thou',
 'thriftless',
 'thy',
 'to',
 "totter'd",
 'treasure',
 'trenches',
 'use,',
 'warm',
 'weed',
 'were',
 'when',
 'where',
 'winters',
 'within',
 'worth',
 "youth's"}

In [ ]:
word_to_ix

{'This': 0,
 'much': 1,
 'on': 2,
 'thine!': 3,
 'thou': 4,
 'held:': 5,
 'If': 6,
 'weed': 7,
 'beauty': 8,
 'an': 9,
 'make': 10,
 'field,': 11,
 'succession': 12,
 'thriftless': 13,
 'worth': 14,
 'trenches': 15,
 'own': 16,
 'thine': 17,
 'in': 18,
 'and': 19,
 'all': 20,
 'count,': 21,
 'Thy': 22,
 "beauty's": 23,
 'where': 24,
 'deep': 25,
 'asked,': 26,
 'Were': 27,
 'sum': 28,
 "'This": 29,
 'lusty': 30,
 'mine': 31,
 "feel'st": 32,
 'it': 33,
 'Shall': 34,
 'Where': 35,
 'made': 36,
 'cold.': 37,
 'child': 38,
 'answer': 39,
 'besiege': 40,
 'to': 41,
 'livery': 42,
 'Proving': 43,
 'Then': 44,
 'by': 45,
 'being': 46,
 'praise': 47,
 'treasure': 48,
 'use,': 49,
 'see': 50,
 'And': 51,
 'small': 52,
 'shame,': 53,
 'forty': 54,
 'days;': 55,
 'winters': 56,
 'all-eating': 57,
 'eyes,': 58,
 'my': 59,
 'when': 60,
 'brow,': 61,
 "youth's": 62,
 'couldst': 63,
 'praise.': 64,
 'Will': 65,
 'fair': 66,
 'To': 67,
 'lies,': 68,
 'How': 69,
 'thy': 70,
 'old': 71,
 'be': 72,
 'wit

In [ ]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [ ]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(100):
    total_loss = 0
    for context, targets in ngrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        model.zero_grad()
        log_probs = model(context_idxs)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[targets]], dtype=torch.long))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

[521.5907504558563, 519.1025950908661, 516.6299440860748, 514.1736812591553, 511.7308552265167, 509.30059480667114, 506.88173270225525, 504.4736969470978, 502.07710313796997, 499.6898684501648, 497.31252431869507, 494.94384121894836, 492.5830092430115, 490.22913002967834, 487.88218688964844, 485.5404107570648, 483.2023570537567, 480.8691072463989, 478.5382821559906, 476.20898938179016, 473.882116317749, 471.556667804718, 469.23264169692993, 466.9091286659241, 464.5848286151886, 462.2585201263428, 459.9307208061218, 457.6008200645447, 455.2685670852661, 452.9346661567688, 450.5985515117645, 448.25821471214294, 445.9134647846222, 443.5656020641327, 441.21330761909485, 438.8544807434082, 436.49055576324463, 434.1190792322159, 431.74177956581116, 429.3565707206726, 426.96392488479614, 424.56172811985016, 422.15298092365265, 419.73607301712036, 417.30966424942017, 414.875079035759, 412.4304469823837, 409.97597229480743, 407.5088268518448, 405.0322141647339, 402.5434663295746, 400.0426104068

In [ ]:
print(model.embeddings.weight[word_to_ix["beauty"]])

tensor([ 1.3974, -0.8213, -0.9506,  1.1082,  0.6263,  0.2079, -2.6301,  0.8581,
        -2.1351, -0.6569], grad_fn=<SelectBackward0>)


In [ ]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

In [ ]:
CONTEXT_SIZE = 2
EMDEDDING_DIM = 100

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [ ]:
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

In [ ]:
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
data[:1]

[(['We', 'are', 'to', 'study'], 'about')]

In [ ]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)


    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

In [ ]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

In [ ]:
#TESTING
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

In [ ]:
#Print result
print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['People', 'create', 'to', 'direct']

Prediction: programs
